In [1]:
import json
import nltk 
import string
import copy
import random
from os import listdir
import pickle


import pandas as pd
import numpy as np

import string
import re

from nltk.stem.porter import *
from nltk.corpus import stopwords
from TurkishStemmer import TurkishStemmer


from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import classification_report
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier,LinearRegression
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score

from pos_tagger import tag

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
# there are four files with dataset, because I collected data in different times
# they are dictionaries, keys are links to news and text of the news is value

text_store=open("article_text_pos_1063_57531copy.json","r")
article_text_dict_positive = json.load(text_store)
text_store.close()
# '3815', "text"

# new negative samples from BBC are :
text_store=open("article_text_neg_bbc_iter1_07272.json","r")
iter1_BBC_text_dict_neg = json.load(text_store)
text_store.close()

text_store=open("iter2_text_neg_bbc_12981.json","r")
iter2_BBC_text_dict_neg = json.load(text_store)
text_store.close()

text_store=open("iter1_article_text_neg_CNN_08109.json","r")
iter1_CNN_neg_text = json.load(text_store)
text_store.close()




In [3]:
toremove=[]
for asd in article_text_dict_positive.items():
    if len(asd[1])<100:
        toremove.append(asd[0])
for keys in toremove:
    del article_text_dict_positive[keys]

In [4]:
print("",len(iter1_BBC_text_dict_neg))
print("",len(iter2_BBC_text_dict_neg))
print("",len(iter1_CNN_neg_text))
print("",len(article_text_dict_positive))

1038
1043
948
1037


In [5]:
#those stop words to be removed
my_stopwords = stopwords.words('turkish')
# those are features that are not related to hate-speech so they will be removed
my_stopwords.extend(['facebook', 'telif','hakkı', 'telif hakkı','dr','bbc','bır','dan', 'den',
                    'karsı','twitter',"caption","image",
                    "image caption","tiklayin",'copyright',
                     'yayın','sayfa','no',
                     "tüm","hakları",'saklıdır','copyright'
                    "sayfa","world","world service","yeni akit","gazetesihalkalı",
                  "tüm hakları saklıdır","hakları saklıdır","hakları",'saklıdır',
                    "yayın","sayfa","no",'ve',"cnn","aa","destekyeniakitcomtr","httpyeniakitcomtr"])

# some websites tends to use special characters too much, remove them all
removethose='“’‘”•.,\'\"!;@?())'
removethose+='*'+ '0'+ ':'+ ']'+ '_'+'$'+ '['+'{'+ '}'+ '»'
removethose+='©'+'-'+"1234567890"+"&"+"—"+"/"+"|"+"="+">"+"…"+"%"+"′"

removethose=['“', '’', '‘', '”', '•', '.', ',', "'", '"', '!', 
             ';', '@', '?', '(', ')', ')', '*', '0', ':', ']',
             '_', '$', '[', '{', '}', '»', '©', '-', '1', '2', 
             '3', '4', '5', '6', '7', '8', '9', '0', '&', '—', 
             '/', '|', '=', '>', '…', '%', '′','€', '¥','£','›',
             '¼','<','¨','‏','­','–','#','+']

lowercase=' abcdefghijklmnoprstuvyzğöıüşç'
# Turkish stemmer from https://github.com/otuncelli/turkish-stemmer-python
stemmer = TurkishStemmer()

# this function lowers text, 
# then removes characters such as @ or » because
# those are used in some resources of news more than others and
# also corrects some characters such as 'i̇' which
# does not exists in Turkish but found in the text
# due to encoding or scraping errors
def pre_process_clean_old(text):
    text=text.lower()
    text=text.strip()
    for rmt in removethose:
        text=text.replace(rmt,"")
    text=text.replace('i̇','i').replace('î','i').replace('â','a').replace('á','a')
    text=text.replace('ū','ü').replace('û','u')
    text=text.replace('è','e').replace('é','e').replace('ê','e')
    return text

def pre_process_clean(text):
    text=text.lower()
    text=text.strip()
    text=text.replace('i̇','i').replace('î','i').replace('â','a').replace('á','a')
    text=text.replace('ū','ü').replace('û','u')
    text=text.replace('è','e').replace('é','e').replace('ê','e')
    result=""
    for thechar in text:
        if (thechar in lowercase):
            result+=thechar
    return result

def pre_process_stopwords(text):
    text=nltk.word_tokenize(text.lower().strip())
    text=filter(lambda x: x not in my_stopwords,text )
    text=" ".join(text)
    return text

# There are different tokenization functions

# first one is tokenizes with nltk, lowers text 
# and applies stemming to each word
def tokenize_stem(text):
    """Removes punctuation & excess whitespace, sets to lowercase,
    and stems text. Returns a list of them"""
    text=nltk.word_tokenize(text.lower().strip())
    tokens = [stemmer.stem(t) for t in text]
    return tokens

# lowers text, uses tagging library's tokenization 
# NO-NEED
# def tokenize(text):
#     text=text.lower().strip()
#     tokens=[]
#     tags=tag(text)
#     for a_tag in tags:
#         if a_tag[0] not in my_stopwords:
#             tokens.append(a_tag[0])
#     return tokens

# lowers text
# generates part of speech tags  
# returns "word+tag" 
# tokenization made with tagging library
def pos_tokenize(text):
    text=text.lower().strip()
    tokens=[]
    tags=tag(text)
    for a_tag in tags:
        tokens.append(a_tag[0]+"+"+a_tag[1])
    return tokens

# lowers text
# splits text to it's characters
def char_tokenize(text):
    text = text.lower().strip()
    tokens = [t for t in text]
    return tokens



In [6]:
# list of feature functions to be tested 
features={"Pos_tags":pos_tokenize,"characters":char_tokenize,"tokenize":nltk.word_tokenize,"tokenize_stem":tokenize_stem}


In [7]:
# this function runs experiments with given data and clasifier then prints results
def run_experiment_w_features(X_train,X_test,y_train,y_test,feature,n_range,theclassifiers,vectorizer=None):
    if vectorizer==None:
        vectorizer = TfidfVectorizer(
            tokenizer= features[feature],
#             cleaning itself
#             preprocessor=word_process_clean,
            ngram_range=n_range,
            use_idf=True,
            min_df=0.003,
            norm=None, )
        
    
    X_train = vectorizer.fit_transform(X_train).toarray()
    X_test=vectorizer.transform(X_test).toarray()

#     y=Y[:]
    if type(theclassifiers)!=list:
        theclassifiers=[theclassifiers]
    for theclassifier in theclassifiers:
#       initialise classifier
        clf= theclassifier()
#     create model with tranining data
        model = clf.fit(X_train, y_train)
#     predict test set
        y_preds = model.predict(X_test)
#     create the report
        report = classification_report( y_test, y_preds )
        
#     find name of the classifier for printing
        match=re.search(r"\.([A-z]*)'>",str(theclassifier))
        match=match.group(1)

        result_text="\033[1m Performance report of \033[0m \033[92m" + feature +"\033[0m "
        count=0
        for i in range(n_range[0],n_range[1]+1):
            if count!=0:
                result_text+=" and "
            result_text=result_text+"\033[91m"+str(i)+"-gram\033[0m"
            count+=1
        result_text+= " with \033[94m"+match+"\033[0m"
    #     print("Performance report of {} {}-gram".format(feature,n_range[1]))
        print (result_text)
#         prnt_scores(report)
        print(report)
        print(accuracy_score( y_test, y_preds))
#     return y_preds,y_test


In [8]:
# pre process text, remove stop words etc
for a_dict in [article_text_dict_positive,iter1_BBC_text_dict_neg,iter2_BBC_text_dict_neg,
iter1_CNN_neg_text]:
    for key,value in a_dict.items():
        a_dict[key]=pre_process_stopwords(pre_process_clean(value))


In [9]:
# for key,value in article_text_dict_positive.items():
#     print (key,value)
#     break

In [25]:
pos_examp=[]
neg_examp_train=[]
neg_examp_test=[]

print("pos examples",len(article_text_dict_positive))
print("BBC",len(iter2_BBC_text_dict_neg))
print("CNN",len(iter1_CNN_neg_text))

# some articles includes short text 
for artc in article_text_dict_positive.values():
    if len(artc)>200:
        pos_examp.append((artc,1))

for artc in iter2_BBC_text_dict_neg.values():
    if len(artc)>200:
        neg_examp_train.append((artc,0))

for artc in iter1_CNN_neg_text.values():
    if len(artc)>200:
        neg_examp_test.append((artc,0))

print("pos examples",len(pos_examp))
print("BBC",len(pos_examp))
print("CNN",len(neg_examp_test))

# combine positive and negative samples then shuffle
# XY=pos_examp+neg_examp_train
random.seed(a=2)
random.shuffle(pos_examp)
random.shuffle(neg_examp_train)
random.shuffle(neg_examp_test)

percentage=0.8
cut_point=int(len(pos_examp)*percentage)
train_pos=pos_examp[:cut_point]
test_pos=pos_examp[cut_point:]

cut_point=int(len(neg_examp_train)*percentage)
train_neg=neg_examp_train[:cut_point]
cut_point=int(len(neg_examp_test)*percentage)
test_neg=neg_examp_test[cut_point:]

print("train_pos",len(train_pos),"test_pos",len(test_pos))
print("train_neg",len(train_neg),"test_neg",len(test_neg))

XY_train=train_pos+train_neg
random.shuffle(XY_train)
X_train=[k[0] for k in XY_train]
Y_train=[k[1] for k in XY_train]

XY_test=test_pos+test_neg
random.shuffle(XY_test)
X_test=[k[0] for k in XY_test]
Y_test=[k[1] for k in XY_test]


print("X_train",len(X_train),"Y_train",len(Y_train),"X_test",len(X_test),"Y_test",len(Y_test))



pos examples 1037
BBC 1043
CNN 948
pos examples 1036
BBC 1036
CNN 948
train_pos 828 test_pos 208
train_neg 824 test_neg 190
X_train 1652 Y_train 1652 X_test 398 Y_test 398


1 emniyet kaynaklarına göre pkk terör örgütünün türkiye kırsalında yaklaşık bin teröristi bulunduğu yurtdışında eli silahlı çeşitli yaş gruplarında binin üzerinde örgüt üyesi olduğu belirtiliyor
2 canlı sayfasında güncellemeler anda durdurulmuştur
2 canlı sayfasında güncellemeler anda durdurulmuştur
2 canlı sayfasında güncellemeler anda durdurulmuştur
2 captioneski başbakan yoshiro mori iki dünya şampiyonluğu bulunan buz patencisi mao asada kritik anlarda düşüyor demişti
2 canlı sayfasında güncellemeler anda durdurulmuştur
2 canlı sayfasında güncellemeler anda durdurulmuştur
2 canlı sayfasında güncellemeler anda durdurulmuştur
2 canlı sayfasında güncellemeler anda durdurulmuştur
2 canlı sayfasında güncellemeler anda durdurulmuştur
2 canlı sayfasında güncellemeler anda durdurulmuştur
2 canlı sayfasında güncellemeler anda durdurulmuştur
2 paylaşdış linkler yeni bir pencerede açılacakbu linki kopyalapaylaşma hakkındadış linkler yeni bir pencerede açılacakpaylaşma menüsünü kapat
12


In [12]:
classifiers_list=[LinearSVC,MultinomialNB,RandomForestClassifier,
                  AdaBoostClassifier]
tryfeatures={"tokenize":nltk.word_tokenize,"tokenize_stem":tokenize_stem,"Pos_tags":pos_tokenize,"characters":char_tokenize}



In [13]:
for clasifier in classifiers_list:
    for feature in tryfeatures:
        for i in range(1,4):
            for k in range(i,4):
                run_experiment_w_features(X_train[:],X_test[:],Y_train[:],Y_test[:],feature,(i,k),clasifier)


/anaconda3/envs/HPCenv/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


 Performance report of  tokenize 1-gram with LinearSVC
              precision    recall  f1-score   support

           0       0.94      0.70      0.80       190
           1       0.78      0.96      0.86       208

   micro avg       0.84      0.84      0.84       398
   macro avg       0.86      0.83      0.83       398
weighted avg       0.86      0.84      0.83       398

0.8366834170854272


/anaconda3/envs/HPCenv/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


 Performance report of  tokenize 1-gram and 2-gram with LinearSVC
              precision    recall  f1-score   support

           0       0.96      0.69      0.80       190
           1       0.77      0.97      0.86       208

   micro avg       0.84      0.84      0.84       398
   macro avg       0.87      0.83      0.83       398
weighted avg       0.86      0.84      0.83       398

0.8366834170854272


/anaconda3/envs/HPCenv/lib/python3.7/site-packages/sklearn/svm/base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


...

 Performance report of  characters 2-gram and 3-gram with LinearSVC
              precision    recall  f1-score   support

           0       0.95      0.64      0.77       190
           1       0.75      0.97      0.85       208

   micro avg       0.81      0.81      0.81       398
   macro avg       0.85      0.81      0.81       398
weighted avg       0.85      0.81      0.81       398

0.8140703517587939
 Performance report of  characters 3-gram with LinearSVC
              precision    recall  f1-score   support

           0       0.97      0.66      0.79       190
           1       0.76      0.98      0.86       208

   micro avg       0.83      0.83      0.83       398
   macro avg       0.87      0.82      0.82       398
weighted avg       0.86      0.83      0.82       398

0.8291457286432161
 Performance report of  tokenize 1-gram with MultinomialNB
              precision    recall  f1-score   support

           0       0.94      0.86      0.90       190
           1   

 Performance report of  Pos_tags 3-gram with MultinomialNB
              precision    recall  f1-score   support

           0       0.76      0.35      0.48       190
           1       0.60      0.90      0.72       208

   micro avg       0.64      0.64      0.64       398
   macro avg       0.68      0.62      0.60       398
weighted avg       0.68      0.64      0.60       398

0.635678391959799
 Performance report of  characters 1-gram with MultinomialNB
              precision    recall  f1-score   support

           0       0.73      0.77      0.75       190
           1       0.78      0.75      0.76       208

   micro avg       0.76      0.76      0.76       398
   macro avg       0.76      0.76      0.76       398
weighted avg       0.76      0.76      0.76       398

0.7587939698492462
 Performance report of  characters 1-gram and 2-gram with MultinomialNB
              precision    recall  f1-score   support

           0       0.84      0.87      0.86       190
        

 Performance report of  tokenize_stem 3-gram with RandomForestClassifier
              precision    recall  f1-score   support

           0       0.74      0.29      0.42       190
           1       0.58      0.91      0.71       208

   micro avg       0.61      0.61      0.61       398
   macro avg       0.66      0.60      0.56       398
weighted avg       0.66      0.61      0.57       398

0.6130653266331658
 Performance report of  Pos_tags 1-gram with RandomForestClassifier
              precision    recall  f1-score   support

           0       0.84      0.83      0.84       190
           1       0.85      0.86      0.85       208

   micro avg       0.85      0.85      0.85       398
   macro avg       0.85      0.85      0.85       398
weighted avg       0.85      0.85      0.85       398

0.8467336683417085
 Performance report of  Pos_tags 1-gram and 2-gram with RandomForestClassifier
              precision    recall  f1-score   support

           0       0.81      0.81

 Performance report of  tokenize 3-gram with AdaBoostClassifier
              precision    recall  f1-score   support

           0       0.89      0.08      0.15       190
           1       0.54      0.99      0.70       208

   micro avg       0.56      0.56      0.56       398
   macro avg       0.72      0.54      0.43       398
weighted avg       0.71      0.56      0.44       398

0.5577889447236181
 Performance report of  tokenize_stem 1-gram with AdaBoostClassifier
              precision    recall  f1-score   support

           0       0.83      0.61      0.70       190
           1       0.71      0.88      0.79       208

   micro avg       0.75      0.75      0.75       398
   macro avg       0.77      0.74      0.74       398
weighted avg       0.77      0.75      0.75       398

0.7512562814070352
 Performance report of  tokenize_stem 1-gram and 2-gram with AdaBoostClassifier
              precision    recall  f1-score   support

           0       0.82      0.63      0

 Performance report of  characters 3-gram with AdaBoostClassifier
              precision    recall  f1-score   support

           0       0.91      0.45      0.60       190
           1       0.66      0.96      0.78       208

   micro avg       0.72      0.72      0.72       398
   macro avg       0.78      0.70      0.69       398
weighted avg       0.78      0.72      0.69       398

0.7160804020100503


In [14]:
slow_classifiers_list=[DecisionTreeClassifier,GaussianNB,MLPClassifier]
for clasifier in slow_classifiers_list:
    for feature in tryfeatures:
        for i in range(1,4):
            run_experiment_w_features(X_train[:],X_test[:],Y_train[:],Y_test[:],feature,(i,i),clasifier)

 Performance report of  tokenize 1-gram with DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.76      0.61      0.68       190
           1       0.70      0.83      0.76       208

   micro avg       0.72      0.72      0.72       398
   macro avg       0.73      0.72      0.72       398
weighted avg       0.73      0.72      0.72       398

0.7236180904522613
 Performance report of  tokenize 2-gram with DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.64      0.47      0.54       190
           1       0.61      0.75      0.67       208

   micro avg       0.62      0.62      0.62       398
   macro avg       0.62      0.61      0.61       398
weighted avg       0.62      0.62      0.61       398

0.6180904522613065
 Performance report of  tokenize 3-gram with DecisionTreeClassifier
              precision    recall  f1-score   support

           0       0.71      0.27      0.39      

 Performance report of  Pos_tags 2-gram with GaussianNB
              precision    recall  f1-score   support

           0       0.78      0.55      0.64       190
           1       0.68      0.86      0.76       208

   micro avg       0.71      0.71      0.71       398
   macro avg       0.73      0.70      0.70       398
weighted avg       0.73      0.71      0.70       398

0.7110552763819096
 Performance report of  Pos_tags 3-gram with GaussianNB
              precision    recall  f1-score   support

           0       0.83      0.21      0.33       190
           1       0.57      0.96      0.72       208

   micro avg       0.60      0.60      0.60       398
   macro avg       0.70      0.58      0.52       398
weighted avg       0.69      0.60      0.53       398

0.6005025125628141
 Performance report of  characters 1-gram with GaussianNB
              precision    recall  f1-score   support

           0       0.72      0.70      0.71       190
           1       0.73      

In [15]:
# y_preds,y_test=run_experiment_w_features(X_train[:],X_test[:],Y_train[:],Y_test[:],"tokenize",(3,3),DecisionTreeClassifier)





In [16]:
# count=0
# for i,k in enumerate(y_preds):
#     if k!=y_test[i]:
#         count+=1
# print(count)
# print(len(y_preds))
    

In [17]:
# 208/398

In [18]:
# # y_preds[100:200]
# y_test[0:110]

In [ ]:
# y_test[-10:]

In [ ]:
# arandom=[0,0,1,1,1]
# random.shuffle(arandom)


In [ ]:
# arandom